In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm

In [2]:
dataset_path = "/home/ns/robosuite/collects/1705874644_525442/demo101_jan21_image_group.hdf5" 
dataset_path = "/home/ns/collect_robomimic_demos/Elbert/NutAssemblySquare_01_26_2024_06_52PM_elbert/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/Elbert/PickPlaceCan_01_26_2024_06_40PM_Elbert/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/Brooke/PickPlaceCan_01_26_2024_02_19PM_Brooke/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/processed/lift_mixed_v0.hdf5"

# dataset_path = "/home/ns/collect_robomimic_demos/sojib_can_expert/PickPlaceCan_01_30_2024_02_33PM_sojib/demo_image.hdf5"
# dataset_path = "/home/ns/collect_robomimic_demos/sojib_square_expert/NutAssemblySquare_02_01_2024_06_38PM_sojib/square_mix_v0.hdf5"
# dataset_path = "/home/ns/collect_robomimic_demos/sojib_can_expert/PickPlaceCan_01_30_2024_01_45PM_sojib/can_mix_v0.hdf5"

dataset_path = "/home/ns/collect_robomimic_demos/processed/demo_image_sub.hdf5"

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())

In [4]:
lengths=[]
demos_minmax={}
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

    action=f['data'][demo_name]['actions']
    action=np.array(action) 
    demos_minmax[demo_name] = (np.min(action, axis=0), np.max(action, axis=0))


lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))
print('Median length: ', np.median(lengths))

print()

for key in f['mask'].keys():
    print(key, f['mask'][key])

Number of demos:  200
Max length:  800
Min length:  116
Mean length:  459.275
Median length:  255.0

expert <HDF5 dataset "expert": shape (100,), type "|S16">
failed <HDF5 dataset "failed": shape (100,), type "|S16">


In [5]:
f.keys()

<KeysViewHDF5 ['data', 'mask']>

In [6]:
# del f['mask']['train']
# del f['mask']['valid']

In [7]:
np.set_printoptions(precision=4, suppress=True)

In [8]:
mins,maxs=[],[]
for demo_name in demos_minmax.keys():
    min,max= demos_minmax[demo_name]
    mins.append(min)
    maxs.append(max)

mins=np.min(mins, axis=0)
maxs=np.max(maxs, axis=0) 
print('mins: ', mins)
print('maxs: ', maxs)

mins:  [-0.6051 -0.6037 -1.1138  0.      0.     -0.45   -1.    ]
maxs:  [1.2542 0.3913 1.0814 0.     0.     0.45   1.    ]


In [9]:
is_normalized='mins' in f.keys()
print(f'is_normalized: {is_normalized}')

if is_normalized:
    print('mins: ', np.array(f['mins']))
    print('maxs: ', np.array(f['maxs']))

is_normalized: False


In [10]:
for key in f['mask'].keys():
    # print(key, f['mask'][key])
    demo_names=[b.decode('utf-8') for b in f['mask'][key] ]

    lengths=[]
    demos_minmax={}
    for demo_name in demo_names:
        demo=f['data'][demo_name]
        num_samples=demo.attrs['num_samples']
        lengths.append(num_samples) 
    lengths=np.array(lengths)

    print(f'mask: {key} length-min: {np.min(lengths)} mean: {np.mean(lengths)} max: {np.max(lengths)} unique: {np.unique(lengths).shape}')

mask: expert length-min: 116 mean: 139.51 max: 206 unique: (51,)
mask: failed length-min: 304 mean: 779.04 max: 800 unique: (3,)


In [12]:
# lengths

In [12]:
f.close()

In [14]:
names=[b.decode('utf-8') for b in f['mask']['expert']]
len(names)

10

In [15]:
names

['demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru']